In [6]:
from reconchess import *
import chess.engine
import os
import random


STOCKFISH_ENV_VAR = 'C:/Users/Rober/Desktop/AI/Assignment/stockfish/stockfish.exe'

In [13]:
class MyAgent(Player):
    def __init__(self):
        self.board = None
        self.color = None
        self.possible_states = None

        # make sure stockfish environment variable exists
        if STOCKFISH_ENV_VAR not in os.environ:
            raise KeyError(
                'TroutBot requires an environment variable called "{}" pointing to the Stockfish executable'.format(
                    STOCKFISH_ENV_VAR))

        # make sure there is actually a file
        stockfish_path = os.environ[STOCKFISH_ENV_VAR]
        if not os.path.exists(stockfish_path):
            raise ValueError('No stockfish executable found at "{}"'.format(stockfish_path))

        # initialize the stockfish engine
        self.engine = chess.engine.SimpleEngine.popen_uci(stockfish_path, setpgrp=True)

    def handle_game_start(self, color, board, opponent_name):
        self.board = board
        self.color = color

        if (color) :
            self.possible_states = {board.fen()}
        else :
            #every opening move white can do
            pseudo_legal_moves = list(board.pseudo_legal_moves)
            for move in pseudo_legal_moves:
                board.push(move)
                self.possible_states.add(board.fen())
                board.pop()

    def handle_opponent_move_result(self, captured_my_piece, capture_square):
        new_states = {}
        if (captured_my_piece) :
            # remove any state where that capture was impossible and add any states where the move was possible
            for state in self.possible_states:
                board = chess.Board(state)
                pseudo_legal_moves = list(board.pseudo_legal_moves)
                attack_on_tile = [move for move in pseudo_legal_moves if move.uci()[-2:] == capture_square]  
                for move in attack_on_tile:
                    board.push(move)
                    new_states.add(board.fen())
                    board.pop()
        else :
            #make every move an opponent can make without capture
            for state in self.possible_states:
                board = chess.Board(state)
                pseudo_legal_moves = list(board.pseudo_legal_moves)
                attack_on_tile = [move for move in pseudo_legal_moves if move.uci()[-2:] != capture_square]  
                for move in attack_on_tile:
                    board.push(move)
                    new_states.add(board.fen())
                    board.pop()
        self.possible_states = new_states

    def choose_sense(self, sense_actions, move_actions, seconds_left):
        #uniform random sense
        return random.choice(sense_actions)
    
    def check_valid_state(state, sense_result):
        # Check if all the squares in a board match the sensed area
        board = chess.Board(state)
        for result in sense_result:
            square = chess.parse_square(result[0])
            on_board = board.piece_at(square)
            if (result[1] is None and on_board is None):
                continue
            if (str(result[1]) == str(on_board)):
                continue
            return False
        return True

    def handle_sense_result(self, sense_result):
        # remove any state where pieces do not match sense
        new_possible_states = {state for state in self.possible_states if not state.check_valid_state(state, sense_result)}
        self.possible_states = new_possible_states

    def choose_move(self, move_actions, seconds_left):
        # stockfish get most popular move
        possible_moves = []
        N = len(self.possible_states)
        run_time = 10/N
        for state in self.possible_states:
            board = chess.Board(state)
            pseudo_legal_moves = list(board.pseudo_legal_moves)
            best_move = None
            for move in pseudo_legal_moves:
                square = chess.parse_square(move.uci()[-2:])
                if (str(board.piece_at(square)).lower() == "k" ):
                    best_move = move.uci()
            if (best_move is None):
                try:
                    board.clear_stack()
                    result = engine.play(board, chess.engine.Limit(time=run_time))
                    best_move = result.move
                except chess.engine.EngineTerminatedError:
                    print('Stockfish Engine died')
                except chess.engine.EngineError:
                    print('Stockfish Engine bad state at "{}"'.format(board.fen()))
            possible_moves.append(best_move)
        move_frequency = {move: possible_moves.count(move) for move in possible_moves}
        possible_moves = sorted(move_frequency, key=lambda move: (-move_frequency[move], move.uci()))

        best_move = possible_moves[0]

        return best_move

    def handle_move_result(self, requested_move, taken_move, captured_opponent_piece, capture_square):
        # I don't think it matters what requested_move was 
        new_states = {}
        if (captured_opponent_piece) :
            # add every state when a capture was possible (if a piece was taken based on actual move)
            move = taken_move.uci()[:-2]
            for state in self.possible_states:
                board = chess.Board(state)
                if (move == capture_square and board.is_capture(taken_move)) :
                    board.push(taken_move)
                    new_states.add(board.fen())
        else:
            # add every state where no capture took place
            move = taken_move.uci()[:-2]
            for state in self.possible_states:
                board = chess.Board(state)
                if (not board.is_capture(taken_move)) :
                    board.push(taken_move)
                    new_states.add(board.fen())
        self.possible_states = new_states

    def handle_game_end(self, winner_color, win_reason, game_history):
        self.engine.quit()

sense_result =
[(34, None), (35, None), (36, None), (26, None), (27, None), (28, None), (18, None), (19, None), (20, None)]
[(34, None), (35, None), (36, None), (26, None), (27, None), (28, None), (18, None), (19, None), (20, None)]
[(34, None), (35, None), (36, None), (26, None), (27, None), (28, None), (18, None), (19, None), (20, None)]
[(34, None), (35, None), (36, None), (26, None), (27, None), (28, None), (18, None), (19, None), (20, None)]
[(34, None), (35, None), (36, None), (26, None), (27, None), (28, None), (18, None), (19, None), (20, None)]
[(34, None), (35, None), (36, None), (26, None), (27, None), (28, None), (18, None), (19, None), (20, None)]
[(34, None), (35, None), (36, None), (26, None), (27, None), (28, None), (18, None), (19, None), (20, None)]
[(34, None), (35, None), (36, None), (26, None), (27, None), (28, None), (18, None), (19, None), (20, None)]
[(34, None), (35, None), (36, None), (26, None), (27, None), (28, None), (18, None), (19, None), (20, None)]
[(34, None), (35, None), (36, None), (26, None), (27, None), (28, None), (18, None), (19, None), (20, None)]
[(34, None), (35, None), (36, None), (26, None), (27, None), (28, None), (18, None), (19, None), (20, None)]
[(34, None), (35, None), (36, None), (26, None), (27, None), (28, None), (18, None), (19, None), (20, None)]
[(34, None), (35, None), (36, None), (26, None), (27, None), (28, None), (18, None), (19, None), (20, None)]
[(34, None), (35, None), (36, None), (26, None), (27, None), (28, None), (18, None), (19, None), (20, None)]
[(34, None), (35, None), (36, None), (26, None), (27, None), (28, None), (18, None), (19, None), (20, None)]
[(34, None), (35, None), (36, None), (26, None), (27, None), (28, None), (18, None), (19, None), (20, None)]
[(34, None), (35, None), (36, None), (26, None), (27, None), (28, None), (18, None), (19, None), (20, None)]
[(34, None), (35, None), (36, None), (26, None), (27, None), (28, None), (18, None), (19, None), (20, None)]
[(34, None), (35, None), (36, None), (26, None), (27, None), (28, None), (18, None), (19, None), (20, None)]
[(34, None), (35, None), (36, None), (26, None), (27, None), (28, None), (18, None), (19, None), (20, None)]